#  Read & interpolate AC thresholds, convert in thresholds on the tympanic membrane in the free field

8-8-2020 Guido Cattani

In [1]:
from math import log10
from pathlib import Path
import pandas as pd

In [2]:
def read_AC_thresholds():
    f_in = '/media/guido/LACIE/Cingle_Guido/Master/Headband/AC_thresholds.xlsx'
    p_in = Path(f_in)   
    df = pd.read_excel(p_in, sheet_name=0, header=0, nrows=85)
    df = df.drop(['Unnamed: 0'], axis=1)
    type_dict = {'Study_ID': 'str', 'Device': 'str',
             'AC_125':'float32', 'AC_250':'float32', 'AC_500':'float32', 
             'AC_1000': 'float32', 'AC_2000': 'float32', 
             'AC_4000': 'float32', 'AC_8000': 'float32'}
    df = df.astype(type_dict)
    df = df.fillna(pd.NA)
    return df

In [3]:
def read_REDD():
    f_in = '/media/guido/LACIE/Cingle_Guido/Master/constants.xlsx'
    p_in = Path(f_in)   
    df = pd.read_excel(p_in, sheet_name='REDD', header = 0, usecols='E:W', skiprows=[1, 2, 3, 4, 5], nrows=1)
    df = df.fillna(pd.NA)
    sr = pd.Series(df.iloc[0], dtype='float32')
    return sr

In [4]:
def log_f_third():
    frequencies = [125, 160, 200, 250, 315, 400, 500, 630, 800, 1000, 
                   1250, 1600, 2000, 2500, 3150, 4000, 5000, 6300, 8000]

    f = pd.Series(frequencies, dtype = 'int16')                             # array with 1/3 octave band CF
    f_log_third_bands = f.apply(log10)                                      # log version of the same array
    return f_log_third_bands

In [5]:
def change_columns(df):
    n = [0, 1, 2, 5, 8, 11, 14, 17, 20]
    cls = list()
    for col in df.columns: 
        cls.append(col) 
    d = dict()
    for i in range(len(cls)):
        d.update({cls[i] : n[i]})
    df = df.rename(columns=d)
    return df

In [6]:
def interpolation_AC_thresholds(df):
    # interpolation of audiometric threholds to obtain threshold for single 1/3 octave band
    frequencies = [125, 160, 200, 250, 315, 400, 500, 630, 800, 
                   1000, 1250, 1600, 2000, 2500, 3150, 4000, 5000, 6300, 8000] # third bands

    flog = log_f_third()

    idx = pd.RangeIndex(21)
    interpol = pd.Series(idx, dtype='float64')

    interpolated = pd.DataFrame()

    for g in range(len(df.index)):
        thr = pd.Series(df.iloc[g])

        interpol[0] = thr[0]
        interpol[1] = thr[1]
        interpol[20] = thr[20]
        for i in range(6):
            j = 3 * i + 2
            t = 3 * i
            interpol[j] = thr[j]
            interpol[j+1] = thr[j] + (thr[j+3] - thr[j]) * (flog[t+1] - flog[t]) / (flog[t+3] - flog[t])
            interpol[j+2] = thr[j] + (thr[j+3] - thr[j]) * (flog[t+2] - flog[t]) / (flog[t+3] - flog[t])

        interpolated = pd.concat([interpolated, interpol], axis = 1)        # concatenation of lines with data

    interpolated = interpolated.T
    interpolated = interpolated.reset_index()
    interpolated  = interpolated.drop(columns=['index'])

    f_names = [str(f) + '_Hz' for f in frequencies]
    name_columns = ['Study_ID', 'Device'] + f_names
    f_dict=dict()
    for i in range(len(f_names)): f_dict.update({f_names[i]:'int32'})

    interpolated.columns = name_columns
    interpolated = interpolated.fillna(-999)
    interpolated = interpolated.round(0)
    interpolated = interpolated.astype(f_dict)
    interpolated = interpolated.replace(-999, pd.NA)
    return interpolated

In [7]:
def correction_REDD(df):
    info = df[['Study_ID', 'Device']]                              # select columns 0 and 1
    df1 = df.drop(columns=['Study_ID', 'Device'])                   # drop columns 0 and 1
    redd = read_REDD()                                               # call function read-REDD
    corrected = df1.add(redd, axis = 1)                               # add Series with REDD values
    corrected = corrected.round(0)
    corrected = corrected.astype('int')
    corrected = pd.concat([info, corrected], axis=1)                 # add newly columns 0 and 1
    return corrected

In [8]:
act = read_AC_thresholds()
act = change_columns(act)
AC_thr_int = interpolation_AC_thresholds(act)
AC_thr_eardrum = correction_REDD(AC_thr_int)

In [9]:
# write results to xlsx file in Master directory
fout = '/media/guido/LACIE/Cingle_Guido/Master/AC_thr_third.xlsx'
pout = Path(fout)
with pd.ExcelWriter(pout) as writer:
    AC_thr_int.to_excel(writer, sheet_name='AC_thr_interpolated')
    AC_thr_eardrum.to_excel(writer, sheet_name='AC_thr_eardrum')